Week 4 - Deployment Hmwk

In [5]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///croot/scikit-learn_1684954695550/work


In [16]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /home/dan/anaconda3/envs/depl/lib/python3.11/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: 


In [6]:
import pickle
import pandas as pd

In [7]:
with open('lin_reg.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/dan/anaconda3/envs/depl/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dan/anaconda3/envs/depl/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
categorical = ['PULocationID','DOLocationID']

def read_data(colour, year, month):
    """ reads data from the NYC taxi website: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
    takes taxi colour, year, month, configures the data for duration prediction
    that we've been using throughout the course
    """
    df = pd.read_parquet(f'https://d37ci6vzurychx.cloudfront.net/trip-data/{colour}_tripdata_{year}-{month}.parquet')

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    # create artificial ride_id column
    df['ride_id'] = f'{year}/{month}_' + df.index.astype('str')

    return df

In [9]:
df = read_data('yellow','2022','02')

In [10]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
y_pred

array([18.53147321, 23.06562476, 33.68961811, ..., 11.8903939 ,
       15.10267328,  9.46057571])

In [12]:
df_result = pd.DataFrame(df['ride_id']).join(pd.DataFrame(y_pred))
df_result = df_result.rename(columns={0: "prediction"})
df_result

,ride_id,prediction
0,2022/02_0,18.531473
1,2022/02_1,23.065625
2,2022/02_2,33.689618
3,2022/02_3,23.760692
4,2022/02_4,21.492842
...,...,...
2979426,2022/02_2979426,NaN
2979427,2022/02_2979427,NaN
2979428,2022/02_2979428,NaN
2979429,2022/02_2979429,NaN


Question 1: Standard Dev is ~ 5.28

In [13]:
df_result.describe()

,prediction
count,2.860150e+06
mean,1.251321e+01
std,5.280695e+00
min,-7.531430e+00
25%,9.945194e+00
50%,1.066466e+01
75%,1.212965e+01
max,7.901338e+01


Question 2: save it to parquet, the file size is 58M

In [14]:
df_result.to_parquet(
    'results',
    engine='pyarrow',
    compression=None,
    index=False
)

In [15]:
!ls -l -h

total 58M
-rw-r--r-- 1 dan dan  18K Jun 15 07:19 lin_reg.bin
-rw-r--r-- 1 dan dan  58M Jun 15 07:48 results
-rw-r--r-- 1 dan dan 5.4K Jun 15 07:27 ride_duration.ipynb


Question 3: Notebook converted to script by using:

`jupyter nbconvert --to script ride_duration.ipynb`

Question 4: Env needs pandas, sklearn, pyarrow, pickle. I got a "locking failed" error for pickle but pickle is included with base python so I just removed it from my command. Launched the env with:

`pipenv install scikit-learn==1.2.2 pandas pyarrow`

First hash: 065e9673e24e0dc5113e2dd2b4ca30c9d8aa2fa90f4c0597241c93b63130d233

Question 5: getting the mean

In [17]:
df_result['prediction'].mean()

12.5132117649594